In [1]:
from tropical import TCs, clean_files
import pandas as pd

In [2]:
# put the dirs in a yaml file
dirs ={ 'regdir':'/home/b/b382216/scratch/regrid_intake',
        'tmpdir':'/home/b/b382216/scratch/tmpdir_intake',
        'fulldir':'/home/b/b382216/scratch/fullres'}

varlist = ['msl', '10u', '10v', 'tp']

#initial year and month
init_year=2020
init_month=9
init_day=29

#final year and month
end_year=2020
end_month=12
end_day=30

# support variables
hour_freq = 6 #hourly frequency to retrieve data
retrieve_frequency = str(hour_freq)+'h' 

# keyword argument or define a class 
# dicitonary with the original filenames
retrieve_dictionary = {'init_year': init_year, 'init_month': init_month, "init_day": init_day, 
                       'end_year': end_year, 'end_month': end_month, "end_day": end_day,
                       'frequency': retrieve_frequency}

In [3]:
tropical = TCs(paths=dirs, var2store=varlist)


In [ ]:
for tstep in pd.date_range(start=f'{retrieve_dictionary["init_year"]}-{retrieve_dictionary["init_month"]}-{retrieve_dictionary["init_day"]}',  
                        end=f'{retrieve_dictionary["end_year"]}-{retrieve_dictionary["end_month"]}-{retrieve_dictionary["end_day"]}', freq=retrieve_dictionary["frequency"]).strftime('%Y%m%dT%H'):
    tropical.readwrite_from_intake(tstep)
    tropical.run_detect_nodes(tstep)
    clean_files(tropical.tempest_filein)
    tropical.store_detect_nodes(tstep)

In [4]:
tropical.set_time_window(n_days_freq=25, n_days_ext=15)


for block in pd.date_range(start=f'{retrieve_dictionary["init_year"]}-{retrieve_dictionary["init_month"]}-{retrieve_dictionary["init_day"]}', 
                               end=f'{retrieve_dictionary["end_year"]}-{retrieve_dictionary["end_month"]}', freq=str(tropical.n_days_freq)+'D'):
         
        dates_freq, dates_ext = tropical.time_window(block)
        tropical.prepare_stitch_nodes(block, dates_freq, dates_ext)
        tropical.run_stitch_nodes(maxgap='6h')
        tropical.reorder_tracks()
        tropical.store_stitch_nodes(block, dates_freq)

Arguments:
  --in <string> ["full_nodes.txt"] 
  --in_list <string> [""] 
  --in_connect <string> [""] 
  --out <string> ["/home/b/b382216/scratch/tmpdir_intake/tempest_track_20200929-20201023.txt"] 
  --in_fmt <string> ["lon,lat,slp,wind"] 
  --range <double> [8.000000] (degrees)
  --mintime <string> ["54h"] 
  --time_begin <string> [""] 
  --time_end <string> [""] 
  --min_endpoint_dist <double> [0.000000] (degrees)
  --min_path_dist <double> [0.000000] (degrees)
  --maxgap <string> ["6h"] 
  --threshold <string> ["wind,>=,10.0,10;lat,<=,50.0,10;lat,>=,-50.0,10"] [col,op,value,count;...]
  --caltype <string> ["standard"] (none|standard|noleap|360_day)
  --allow_repeated_times <bool> [false] 
  --out_file_format <string> ["gfdl"] (gfdl|csv|csvnohead)
------------------------------------------------------------
Parsing thresholds
..wind greater than or equal to 10.000000 at least 10 time(s)
..lat less than or equal to 50.000000 at least 10 time(s)
..lat greater than or equal to -50.000